# mCT DL-HMC Training & Testing template - baseline V8
Works for both single and multi subjects studies. <br>
Last update (01/28):Modify the google service related code and function

20230123: first version, DLHMC baseline, use point cloud, resize 96x96x64;<br>
20230130: second version, modify the evaluation;<br>
20230203: third version, used for suv norm resized 1s recon.<br>
20230208: ues unique ID for identification, update the evaluation<br>
20230209: test for memory, use normal cache; small encoder from ISBI paper<br>
20230217: use new sampling and data loader;<br>
20230219: data augmentation from Jiazhen;<br>
20230223: output 6 parameters, no FWT;<br>

In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    Spacingd, \
    ToTensord,  \
    DataStatsd, \
    ToDeviced, Affine
from monai.data import list_data_collate
import torch
import pytorch_lightning as pl
from torchsummary import summary
monai.config.print_config()
import sys
sys.path.append(r'../util/python')
import mCTdata_vicra_toolbox
import nibabel; nibabel.imageglobals.logger.setLevel(40)
import time
# New transforms 

sys.path.append(r'../')

from dlhmc.transforms import (
    CreateImageStack,
    ComputeRelativeMotion,
    RandSamplePET,
    ComputeRelativeMotiond,
    CreateImageStackd,
    RandSamplePETd,
)
from dlhmc.utils import Relative_motion_A_to_B_12,RotTransMatrix_6Params
import nibabel 
nibabel.imageglobals.logger.setLevel(40)

from dlhmc.utils.data import (
    concatenate_vicra,
    split_dataset
)

2023-03-01 17:24:01,989 - Created a temporary directory at /tmp/tmpv6p9a0va
2023-03-01 17:24:01,992 - Writing /tmp/tmpv6p9a0va/_remote_module_non_scriptable.py
MONAI version: 1.0.1
Numpy version: 1.23.4
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8271a193229fe4437026185e218d5b06f7c8ce69
MONAI __file__: /home1/tz323/anaconda3/envs/dl-hmc_2301/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.11.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.14.1
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UN

In [2]:
torch.set_num_threads(4)

## Data loading and sampling

In [3]:
# Load available data
#!!! Function get_data & print_info needs update

from mCTdata_summary_toolbox import get_data, print_info

df, df_ready, patients_ready =get_data('dl-hmc-data-analysis-08b527b43bc1.json', ['FPEB'])
print_info('dl-hmc-data-analysis-08b527b43bc1.json', ['FPEB'])

For tracer  FPEB   24  patients out of  24  are ready.
Ready patients are ['DC251_28608', 'DC251_28497', 'DC251_28504', 'DP162_27423', 'DP162_28061', 'DP162_28064', 'EC878_27647', 'EC878_27410', 'EC878_27644', 'JV527_27033', 'JV527_27031', 'JV527_27020', 'KS553_28733', 'KS553_28544', 'KS553_28529', 'MH419_27598', 'MH419_27361', 'MH419_27612', 'MM537_27024', 'MM537_27026', 'MM537_27028', 'RA253_28092', 'RA253_28137', 'RA253_28138']


In [3]:
''' You can also get and visualize information using other functions of the dataset_summary toolbox, such as

-compute_delta_T: compute delta_T for each patient and add it to the summary
-plot_delta_T_all: plot the delta_T computed by compute_delta_T
-delta_T_norm: normalizes delta_T and plots it
-delta_T_smoothed: smoothes delta_T_norm and plots it
-data_analysis_display: displays information about the motion (magnitude, when do the major movements occur, mean and SD of delta_T...)
-average_motion: computes and plot the average delta_T at each second among patients
-make_gif_3Dcloud: make a gif with the 3D cloud data for a list of patients '''

' You can also get and visualize information using other functions of the dataset_summary toolbox, such as\n\n-compute_delta_T: compute delta_T for each patient and add it to the summary\n-plot_delta_T_all: plot the delta_T computed by compute_delta_T\n-delta_T_norm: normalizes delta_T and plots it\n-delta_T_smoothed: smoothes delta_T_norm and plots it\n-data_analysis_display: displays information about the motion (magnitude, when do the major movements occur, mean and SD of delta_T...)\n-average_motion: computes and plot the average delta_T at each second among patients\n-make_gif_3Dcloud: make a gif with the 3D cloud data for a list of patients '

In [3]:
df_ready

,PatientID,Name,InjectionID,UniqueID,date,Weight_kg,Injtime,dose_mCi,tracer,isotope,protocol,Diagnosis,Secondary_Diagnosis,pstate,MotMag_entire_EM_mm,Status (Done/Running),MCCOD QA,Training/Testing_TZ
0,DC251,"DC251, human",28608,DC251_28608,2021/11/10,86.1,12:29:06,1.74,FPEB,F-18,fpeb_sam,Healthy_Control,,baseline,,Done,Good,Training
1,DC251,"DC251, human",28497,DC251_28497,2021/10/19,88,13:48:56,4.75,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_one,,Done,Good,Training
2,DC251,"DC251, human",28504,DC251_28504,2021/10/20,88,11:59:18,4.73,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_two,,Done,Good; No motion,Training
3,DP162,"DP162, human",27423,DP162_27423,2021/5/26,68,13:44:39,4.28,FPEB,F-18,fpeb_sam,Healthy_Control,,baseline,,Done,Good,Training
4,DP162,"DP162, human",28061,DP162_28061,2021/8/24,70,13:50:21,3.25,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_one,,Done,Not perfect,Training
5,DP162,"DP162, human",28064,DP162_28064,2021/8/25,70,11:49:06,4.72,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_two,,Done,Not perfect; Major movements,Testing
6,EC878,"EC878, human",27647,EC878_27647,2021/6/30,106,11:36:52,4.48,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_two,,Done,Good.,Training
7,EC878,"EC878, human",27410,EC878_27410,2021/5/25,106.5,13:56:51,4.45,FPEB,F-18,fpeb_sam,Healthy_Control,,baseline,,Done,Wrong Vicra,Testing
8,EC878,"EC878, human",27644,EC878_27644,2021/6/29,106,13:52:01,4.42,FPEB,F-18,fpeb_sam,Healthy_Control,,post-dose_one,,Done,Good; No motion,Training
9,JV527,"JV527, human",27033,JV527_27033,2021/4/14,65.4,14:58:53,4.68,FPEB,F-18,fpeb_sam,Healthy_Control,,baseline,,Done,Good,Training


In [4]:
tr_set=['DP162_27423']
test_set=['KS553_28544']

In [5]:
# Choose training and testing patients

tr_set=['DC251_28608','DP162_27423','EC878_27647','EC878_27644','JV527_27033','JV527_27031','KS553_28544','KS553_28733','MH419_27361','MH419_27598','MM537_27024','MM537_27026','RA253_28092','RA253_28138']
test_set=['KS553_28544']

In [5]:
inters=set(tr_set) & set(test_set)
if len(inters)>0:
    print('Careful, there\'s data contamination between the training and testing sets. The duplicates are: ', inters)

In [6]:
# Split and sample the training set

from mCTdata_summary_toolbox import compute_delta_T 
from mCTdata_prep_toolbox import delta_T_magnitude, Relative_motion_A_to_B_12, build_legal_dataset, deal_dataframe, clean_df
from sampling_toolbox import data_split_sample, add_T_deltaT

# Get the summaries

summaries, delta_T_all=compute_delta_T(['FPEB'], tr_set)

In [7]:
#modify 3D clouds to recon
df_index=0
for summary in summaries: 
    summary_dict = summary.to_dict('records')
    for i in range(len(summary_dict)):
        x = summary_dict[i]['ThreeD_Cloud'].find('3D_Clouds_nii')
        fn_cloud1 = summary_dict[i]['ThreeD_Cloud'][0:x] + 'MOLAR_Recon_NAC_nii_suv_resize'
        x = x+13
        y = summary_dict[i]['ThreeD_Cloud'].find('3D_cloud')
        fn_cloud2 =  summary_dict[i]['ThreeD_Cloud'][x:y] + 'recon_suv_rz.nii'
        summary_dict[i]['ThreeD_Cloud'] = fn_cloud1 + fn_cloud2
    summaries[df_index] = pd.DataFrame(summary_dict)
    df_index = df_index + 1


In [8]:
len(summaries)

1

In [10]:
summaries[0].at[1000,'ThreeD_Cloud']

'/data16/public/registration-brain_PET/data/mCT_real/FPEB/20211110_DC251/28608/MOLAR_Recon_NAC_nii_suv_resize/DC251-Human-20211110.44940_EM_4600_4601_nmc_150x150x109_1frame_recon_suv_rz.nii'

In [7]:
num_subsample = 1000 # Number of samples per subject (to be split between training, validation and testing sets)

train_list=list()
val_list=list()
test_list=list()

# Sample training, validation and testing sets and save them in lists
for summary in summaries: 
    summary['VICRA']=concatenate_vicra(summary)

    print('Dataset contains total {:,d} entries'.format(len(summary)))
    df_train, df_val, df_test = split_dataset(summary, num_subsample=num_subsample, test_size_percent=0.1, validation_size_percent=0.1)
    print('Dataset split into 3 subsets:')
    print(' Train set size: {:,d}'.format(len(df_train)))
    print(' Val set size: {:,d}'.format(len(df_val)))
    print(' Test set size: {:,d}'.format(len(df_test)))
    
    train_dict={
        'DataFrame':df_train
    }

    val_dict={
        'DataFrame':df_val
    }
    
    test_dict={
        'DataFrame':df_test
    }
    
    train_list.append(train_dict)
    val_list.append(val_dict)
    test_list.append(test_dict)

Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 800
 Val set size: 100
 Test set size: 100


In [11]:
df_train.columns

Index(['PatientID', 'InjectionID', 'PatientType', 'Listmode', 'ScanStartTime',
       'InjectionTime', 'ScanStartTime_ms', 'InjectionTime_ms', 'ScanStart',
       'ScanEnd', 'ThreeD_Cloud', 'COD_nomask', 'Resampled_Vicra', 'NAC_MOLAR',
       'VC_Timetag', 'VC_11', 'VC_12', 'VC_13', 'VC_14', 'VC_21', 'VC_22',
       'VC_23', 'VC_24', 'VC_31', 'VC_32', 'VC_33', 'VC_34',
       'Total_MC_NAC_MOLAR', 'MATRIX', 'UniqueID', 'T', 'delta_T',
       'relative_T', 'delta_relative_T', 'VICRA'],
      dtype='object')

In [12]:
df_train.head()

,PatientID,InjectionID,PatientType,Listmode,ScanStartTime,InjectionTime,ScanStartTime_ms,InjectionTime_ms,ScanStart,ScanEnd,...,VC_33,VC_34,Total_MC_NAC_MOLAR,MATRIX,UniqueID,T,delta_T,relative_T,delta_relative_T,VICRA
0,RA253,28138,Real,/data4/mct/raw/20210901_RA253/RA253.PT.PET_07_...,11:57:30,11:57:51,43050000,43071000,3600,3601,...,0.999727,-3.295153,/data16/public/registration-brain_PET/data/mCT...,"[0.999876, 0.000152, -0.015752, -0.962667, -0....",RA253_28138,"[-0.962667, 1.003165, -3.295153, 0.98819337941...",0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,"[0.999876, 0.000152, -0.015752, -0.962667, -0...."
1,RA253,28138,Real,/data4/mct/raw/20210901_RA253/RA253.PT.PET_07_...,11:57:30,11:57:51,43050000,43071000,3601,3602,...,0.999730,-3.781309,/data16/public/registration-brain_PET/data/mCT...,"[0.99987, -0.00082, -0.016133, -0.962722, 0.00...",RA253_28138,"[-0.962722, 1.028904, -3.781309, 0.95746868397...",0.241534,"[-0.007721414614703548, 0.017349683226031365, ...",0.241588,"[0.99987, -0.00082, -0.016133, -0.962722, 0.00..."
3,RA253,28138,Real,/data4/mct/raw/20210901_RA253/RA253.PT.PET_07_...,11:57:30,11:57:51,43050000,43071000,3603,3604,...,0.999721,-3.416151,/data16/public/registration-brain_PET/data/mCT...,"[0.99986, -0.001463, -0.016662, -0.983561, 0.0...",RA253_28138,"[-0.983561, 0.982657, -3.416151, 0.96022827104...",0.027566,"[-0.022788077241275184, -0.022595175927928035,...",0.027648,"[0.99986, -0.001463, -0.016662, -0.983561, 0.0..."
6,RA253,28138,Real,/data4/mct/raw/20210901_RA253/RA253.PT.PET_07_...,11:57:30,11:57:51,43050000,43071000,3606,3607,...,0.999739,-3.271562,/data16/public/registration-brain_PET/data/mCT...,"[0.999857, -0.002546, -0.016736, -1.092886, 0....",RA253_28138,"[-1.092886, 0.859727, -3.271562, 0.89043618254...",0.074732,"[-0.12977057353001564, -0.14302961755787524, 0...",0.075208,"[0.999857, -0.002546, -0.016736, -1.092886, 0...."
7,RA253,28138,Real,/data4/mct/raw/20210901_RA253/RA253.PT.PET_07_...,11:57:30,11:57:51,43050000,43071000,3607,3608,...,0.999708,-3.478838,/data16/public/registration-brain_PET/data/mCT...,"[0.999853, -0.002436, -0.016957, -1.048677, 0....",RA253_28138,"[-1.048677, 1.1303, -3.478838, 0.9872792230469...",0.084071,"[-0.08894567789543384, 0.12393470980390309, -0...",0.084075,"[0.999853, -0.002436, -0.016957, -1.048677, 0...."


In [15]:
# df_train.at[2,'ThreeD_Cloud']

In [13]:
len(df_train)

1440

In [8]:
# Set transforms 

SIZE = (96,96,64)

image_key = "ThreeD_Cloud"
meta_keys=['ScanStart','VICRA']

# Need to format as a dictionary for input
# sample_dict = {
#     'DataFrame': df_train,
    # 'PatientID': df_sample['PatientID'].unique()
# }

# Here, we only have one subject of data, so a small example, but more subjects can be added
# to this list.
# sample_data_dict = [sample_dict]

pet_transforms = monai.transforms.Compose([
    CreateImageStackd(keys="DataFrame", image_key=image_key, spatial_size=SIZE),
    RandSamplePETd(keys=image_key, meta_data_key="DataFrame", meta_keys=meta_keys, num_samples=16), 
    ComputeRelativeMotiond(keys=['VICRA_ref','VICRA_mov'], output_key='VICRA_rel'),
    monai.transforms.DeleteItemsd(keys=["DataFrame", "ScanStart", "VICRA_ref", "VICRA_mov", image_key]),
    monai.transforms.ToTensord(
        keys=["ScanStart_ref", "ScanStart_mov", "VICRA_rel"]
    )
])

In [15]:
print(type(train_list))
print(len(train_list))
print(type(train_list[0]))
print(len(train_list[0]))
print(train_list[0].keys())

<class 'list'>
14
<class 'dict'>
1
dict_keys(['DataFrame'])


In [9]:
# Create datasets and data loaders for training, validation and testing sets 

train_ds = monai.data.PersistentDataset(
    data=train_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache_V7/"
#     cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache/"
)

train_loader = torch.utils.data.DataLoader(
    train_ds, 
    batch_size=2, 
    shuffle=True, 
    #num_workers=0, 
    collate_fn=monai.data.list_data_collate
)

val_ds = monai.data.PersistentDataset(
    data=val_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache_V7/"
#     cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache/"
)

val_loader = torch.utils.data.DataLoader(
    val_ds, 
    batch_size=2, 
    shuffle=True, 
    #num_workers=0, 
    collate_fn=monai.data.list_data_collate
)

In [16]:
# Create datasets and data loaders for training, validation and testing sets 

train_ds = monai.data.CacheDataset(
    data=train_list, 
    transform=pet_transforms, 
#     cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache/"
)

train_loader = torch.utils.data.DataLoader(
    train_ds, 
    batch_size=2, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=monai.data.list_data_collate
)

val_ds = monai.data.CacheDataset(
    data=val_list, 
    transform=pet_transforms, 
#     cache_dir="/data16/private/tz323/dev/mCT-dl-hmc/cache/"
)

val_loader = torch.utils.data.DataLoader(
    val_ds, 
    batch_size=4, 
    shuffle=True, 
    num_workers=1, 
    collate_fn=monai.data.list_data_collate
)

Loading dataset:   0%|                                                                                     | 0/14 [00:00<?, ?it/s]/home1/tz323/anaconda3/envs/dl-hmc_2301/lib/python3.9/site-packages/monai/transforms/io/array.py:191: UserWarning: required package for reader ITKReader is not installed, or the version doesn't match requirement.
  warnings.warn(
Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████| 14/14 [01:42<00:00,  7.32s/it]


In [17]:
print(len(train_ds))
print(len(train_loader))

14
7


In [12]:
import time

start_time = time.time()

for batch in train_loader:
    print("keys: ", batch.keys())
    print(batch['ScanStart_ref'])
    print(batch['ScanStart_mov'])
    print(batch['ThreeD_Cloud_ref'].shape)
    print(batch['VICRA_rel'].shape)

stop_time = time.time()
processing_time = stop_time-start_time
print('First epoch processing time: {:.3}s'.format(processing_time))

keys:  dict_keys(['ThreeD_Cloud_ref', 'ThreeD_Cloud_mov', 'ScanStart_ref', 'ScanStart_mov', 'VICRA_rel', 'ScanStart_ref_transforms', 'ScanStart_mov_transforms', 'VICRA_rel_transforms'])
tensor([3601, 3612, 3685, 3923, 3928, 3945, 4016, 4075, 4088, 4089, 4105, 4237,
        4316, 4542, 4600, 4705])
tensor([5199, 4107, 4152, 4731, 4047, 4442, 5267, 4209, 4906, 4513, 4184, 5222,
        4715, 4713, 4897, 5307])
(16, 1, 96, 96, 64)
(16, 12)
First epoch processing time: 4.35s


In [13]:
import time

start_time = time.time()

for batch in val_loader:
    print("keys: ", batch.keys())
    print(batch['ScanStart_ref'])
    print(batch['ThreeD_Cloud_ref'].shape)
    print(batch['VICRA_rel'].shape)

stop_time = time.time()
processing_time = stop_time-start_time
print('First epoch processing time: {:.3}s'.format(processing_time))

keys:  dict_keys(['ThreeD_Cloud_ref', 'ThreeD_Cloud_mov', 'ScanStart_ref', 'ScanStart_mov', 'VICRA_rel', 'ScanStart_ref_transforms', 'ScanStart_mov_transforms', 'VICRA_rel_transforms'])
tensor([3758, 3801, 3863, 3926, 3965, 3994, 4072, 4320, 4360, 4510, 4533, 4555,
        4580, 4594, 4674, 4674])
(16, 1, 96, 96, 64)
(16, 12)
First epoch processing time: 0.91s


## Build network

In [9]:
import torch.nn as nn
from monai.networks.blocks import Convolution

# Encoder + intermediate layer

simple_enc_inter=nn.Sequential()
simple_enc_inter.add_module("conv1", Convolution(
    strides=2,
    spatial_dims=3,
    kernel_size=5,
    in_channels=1,
    out_channels=32,
    adn_ordering="ADN",
    act=("prelu", {"init": 0.2}),
    dropout=0.1,
#     norm=("layer", {"normalized_shape": (10, 10, 10)}),
    padding=(0,0,0)
))
simple_enc_inter.add_module("conv2", Convolution(
    strides=2,
    kernel_size=5,
    spatial_dims=3,
    in_channels=32,
    out_channels=64,
    adn_ordering="ADN",
    act=("prelu", {"init": 0.2}),
    dropout=0.1,
#     norm=("layer", {"normalized_shape": (10, 10, 10)}),
    padding=(0,0,0)
))
simple_enc_inter.add_module("conv3", Convolution(
    strides=2,
    kernel_size=5,
    spatial_dims=3,
    in_channels=64,
    out_channels=128,
    adn_ordering="ADN",
    act=("prelu", {"init": 0.2}),
    dropout=0.1,
#     norm=("layer", {"normalized_shape": (10, 10, 10)}),
    padding=(0,0,0)
))
simple_enc_inter.add_module("intermediate", Convolution(
    strides=2,
    kernel_size=3,
    spatial_dims=3,
    in_channels=128,
    out_channels=16,
    adn_ordering="ADN",
    act=("prelu", {"init": 0.2}),
    dropout=0.1,
#     norm=("layer", {"normalized_shape": (10, 10, 10)}),
    padding=(0,0,0)
))

In [10]:
resl_pix = np.array([3.18191,3.18191,3.45224])
def generate_synthetic_motion(trans_amp=10,rot_amp=5,resl_pix=None):
    if not isinstance(resl_pix,np.ndarray):
        raise TypeError("No voxel size specified")
    tx=np.random.uniform(-trans_amp, trans_amp)# mm
    ty=np.random.uniform(0.0, trans_amp) # cannot move towards beds
    tz=np.random.uniform(-trans_amp, trans_amp)
    rx=np.random.uniform(-rot_amp, rot_amp)# degree
    ry=np.random.uniform(-rot_amp, rot_amp)
    rz=np.random.uniform(-rot_amp, rot_amp)
    T_syn = np.array([tx, ty, tz, rx, ry, rz])

    # convert into monai affine compatible
    T_syn_monai = np.zeros(6, dtype=float)
    T_syn_monai[:3] = T_syn[:3] / resl_pix
    T_syn_monai[3:] = -1 * np.deg2rad(T_syn[3:])
    
    return T_syn, T_syn_monai

In [11]:
class PETRegNet(pl.LightningModule):
    
    def __init__(self, dropout=0.3):
        super().__init__()
        
        self.dropout = dropout
        
        self.simple_enc_inter=simple_enc_inter
        
        self.fwt_layers = torch.nn.Sequential(
            torch.nn.Linear(2, 256),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(256, 512),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(512, 1024),            
        )

        self.regression_layers = torch.nn.Sequential(
            torch.nn.Linear(1024, 512),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(512, 256),
            torch.nn.LeakyReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(256, 6),
        )
        
        monai.utils.set_determinism(seed=42)
        self.simple_enc_inter.apply(self._init_weights)
        self.fwt_layers.apply(self._init_weights)
        self.regression_layers.apply(self._init_weights)
        
        self.loss_function = torch.nn.MSELoss()
        
        self.start_time = time.time()
        self.stop_time = time.time()
        
    def _init_weights(self, m):
        classname = m.__class__.__name__
        if "conv" in classname.lower():
            if hasattr(m, 'weight'):
                torch.nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
#                     torch.nn.init.constant_(m.bias, 0.3)
                    torch.nn.init.xavier_normal_(m.bias[None])
        elif "linear" in classname.lower():
            if hasattr(m, 'weight'):
                torch.nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    # print('conv_bias', (m.bias.shape))
                    torch.nn.init.xavier_normal_(m.bias[None])
        
    def forward(self, x1, x2, x_t):
        y1=self.simple_enc_inter(x1)
        y2=self.simple_enc_inter(x2)

        # FWT weights computation
        x_t=x_t.float()
        x_fwt = self.fwt_layers(torch.transpose(x_t[None],0,1))
        x_fwt=torch.squeeze(x_fwt, dim=1)

        # Encoder and intermediate layer + FWT multiplication
        x_fwt_ref=x_fwt[:,:512]
        x_fwt_ref2=torch.reshape(x_fwt_ref, y1.shape)
        y1=torch.mul(y1,x_fwt_ref2)
        y1_flat=torch.flatten(y1, start_dim=1)#to feed in the regression layers 
        
        x_fwt_mov=x_fwt[:,512:]
        x_fwt_mov=torch.reshape(x_fwt_mov, y2.shape)
        y2=torch.mul(y2,x_fwt_mov)
        y2_flat=torch.flatten(y2, start_dim=1)#to feed in the regression layers 

        #Regression
        y = torch.cat([y1_flat,y2_flat], dim=-1)
        y = self.regression_layers(y)

        return y

    def prepare_data(self):
        # set deterministic training for reproducibility
        monai.utils.misc.set_determinism(seed=42)

    def training_step(self, batch, batch_idx):
        batch_start_time = time.time()
        
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        
        syn_mov_list = list()
        syn_rev_list = list()
        for mov_img, rel_motion in zip(x2, gt_reg):
            # image
            T_syn, T_syn_monai = generate_synthetic_motion(
                trans_amp=10,rot_amp=5,resl_pix=resl_pix)
            syn_mov_img = Affine(
                            rotate_params=tuple(T_syn_monai[3:]),
                            translate_params=tuple(T_syn_monai[:3]),
                            mode = 'bilinear',
                            padding_mode='zeros')(mov_img)[0]
            syn_mov_list.append(syn_mov_img)
            
            # motion
            T_line = RotTransMatrix_6Params(T_syn,2)
            T_matrix = np.reshape(T_line,(3,4))
            T_matrix = np.insert(T_matrix, 3, np.array([0, 0, 0, 1]), axis=0)
            T_matrix_inv = np.linalg.inv(T_matrix)
            
            rel_matrix = np.reshape(rel_motion,(3,4))
            rel_matrix = np.insert(rel_matrix, 3, np.array([0, 0, 0, 1]), axis=0)
            
            syn_matrix = np.dot(rel_matrix, T_matrix_inv)
            syn_line12 = np.reshape(syn_matrix, 16)[0:12]
            syn_line = torch.from_numpy(RotTransMatrix_6Params(syn_line12,1))
            syn_rev_list.append(syn_line)
            
        syn_x2 = torch.stack(syn_mov_list)
        syn_reg = torch.stack(syn_rev_list).float().to(device)
        
        
#         y = self.forward(x1,x2,x_t)
#         loss = self.loss_function(y, gt_reg)
        y = self.forward(x1,syn_x2,x_t)
        loss = self.loss_function(y, syn_reg)    
        
        batch_stop_time = time.time()
        batch_processing_time = batch_stop_time-batch_start_time
        print('Batch {} processing time: {:.3}s'.format(batch_idx,batch_processing_time))
        
        return {"loss": loss}
        
    
    def training_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Train/Loss', avg_loss, self.current_epoch)
        
        self.stop_time = time.time()
        processing_time = self.stop_time-self.start_time
        print('One epoch processing time: {:.3}s'.format(processing_time))
        self.start_time = time.time()
    
    
    def validation_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        #print('gt:',gt_reg)
        i_6_list = list()
        for i_12 in gt_reg:
            i_6 = np.reshape(i_12,12)
            i_6 = torch.from_numpy(RotTransMatrix_6Params(i_6,1))
            i_6_list.append(i_6)
            
        gt_reg_6 = torch.stack(i_6_list).float().to(device)    
        y = self.forward(x1,x2,x_t)
        loss = self.loss_function(y, gt_reg_6)
        #print('y',y)
        #print('gt_reg_6:',gt_reg_6)
        return {"val_loss": loss}

    
    def validation_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Val/Loss', avg_loss, self.current_epoch)

        # Log the value for model checkpoint saving
        self.log('val_loss', avg_loss.item()) #added .item(), otherwise validation wouldn't work 

    def configure_optimizers(self):
        total_params = list(self.simple_enc_inter.parameters()) + list(self.regression_layers.parameters()) + list(self.fwt_layers.parameters())
        opt = torch.optim.Adam(total_params, lr=5e-4)
        scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer=opt, step_size=150, gamma=0.98),
                     'name': 'Learning Rate'}
        return [opt], [scheduler]

In [12]:
from torchinfo import summary
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = PETRegNet().to(device)
#summary(model, [(1,1,96,96,64),(1,1,96,96,64),(1,2)],depth=2)

In [16]:
# for idx, p in enumerate(model.simple_enc_inter.conv1.parameters()):
#     print(p.shape, torch.flatten(p[0,...]))
        
# model.simple_enc_inter.conv1

In [13]:
MODEL_PATH = os.path.join('.','saved_model_recon_V8_multi')
print('MODEL_PATH={}'.format(MODEL_PATH))

MODEL_PATH=./saved_model_recon_V8_multi


## Training

In [ ]:
# Initialise the LightningModule
model = PETRegNet(dropout=0.3)

# Set up loggers and checkpoints
tb_logger = pl.loggers.TensorBoardLogger(
    save_dir=os.path.join(MODEL_PATH,'logs')
)

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(MODEL_PATH),
    filename="PETRegNet-{epoch}-{val_loss:.3f}",
    monitor='val_loss',
    save_last=True,
    save_top_k=2,
    mode='min'
)

# Check for last checkpoint
resume_checkpoint = None
if os.path.exists(os.path.join(MODEL_PATH,'last.ckpt')):
    resume_checkpoint = os.path.join(MODEL_PATH,'last.ckpt')

# Initialise Lightning's trainer.
trainer = pl.Trainer(
    accelerator='gpu',
    devices=[2],
    max_epochs=6000, #10k is generally enough for single-subjct studies
    logger=tb_logger,
    callbacks=[lr_monitor,checkpoint_callback],
    num_sanity_val_steps=1,
    check_val_every_n_epoch=20,
#     resume_from_checkpoint=resume_checkpoint # deprecated since v1.5
)

# Train
trainer.fit(model, train_loader, val_loader, ckpt_path=resume_checkpoint) # previously was trainer.fit(model, train_dataloader=tr_loader, val_dataloaders=val_loader)

2023-02-25 19:33:59,385 - GPU available: True (cuda), used: True
2023-02-25 19:33:59,386 - TPU available: False, using: 0 TPU cores
2023-02-25 19:33:59,387 - IPU available: False, using: 0 IPUs
2023-02-25 19:33:59,389 - HPU available: False, using: 0 HPUs
2023-02-25 19:33:59,393 - Restoring states from the checkpoint path at ./saved_model_recon_V8_multi/last.ckpt
2023-02-25 19:33:59,454 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
2023-02-25 19:33:59,456 - 
  | Name              | Type       | Params
-------------------------------------------------
0 | simple_enc_inter  | Sequential | 1.3 M 
1 | regression_layers | Sequential | 657 K 
2 | loss_function     | MSELoss    | 0     
-------------------------------------------------
2.0 M     Trainable params
0         Non-trainable params
2.0 M     Total params
7.989     Total estimated model params size (MB)
2023-02-25 19:33:59,473 - Restored all states from the checkpoint file at ./saved_model_recon_V8_multi/last.ckpt


Training: 7it [00:00, ?it/s]

Batch 0 processing time: 17.4s
Batch 1 processing time: 0.184s
Batch 2 processing time: 0.191s
Batch 3 processing time: 0.217s
Batch 4 processing time: 0.203s
Batch 5 processing time: 0.197s
Batch 6 processing time: 0.194s
One epoch processing time: 55.2s
Batch 0 processing time: 0.223s
Batch 1 processing time: 0.196s
Batch 2 processing time: 0.196s
Batch 3 processing time: 0.209s
Batch 4 processing time: 0.2s
Batch 5 processing time: 0.178s
Batch 6 processing time: 0.201s
One epoch processing time: 42.1s
Batch 0 processing time: 0.206s
Batch 1 processing time: 0.179s
Batch 2 processing time: 0.219s
Batch 3 processing time: 0.181s
Batch 4 processing time: 0.184s
Batch 5 processing time: 0.223s
Batch 6 processing time: 0.186s
One epoch processing time: 41.0s
Batch 0 processing time: 0.199s
Batch 1 processing time: 0.209s
Batch 2 processing time: 0.198s


## Testing and evaluation

In [14]:
from mCTdata_evaluation_toolbox import build_df_results, show_df_loss, plot_vicra_network, plot_diff_vicra_network, save_synthetic_vicra, print_loss

In [19]:
test_list[0]

{'DataFrame':      PatientID  InjectionID PatientType  \
 24       DC251        28608        Real   
 48       DC251        28608        Real   
 59       DC251        28608        Real   
 72       DC251        28608        Real   
 101      DC251        28608        Real   
 ...        ...          ...         ...   
 1716     DC251        28608        Real   
 1759     DC251        28608        Real   
 1760     DC251        28608        Real   
 1778     DC251        28608        Real   
 1795     DC251        28608        Real   
 
                                                Listmode ScanStartTime  \
 24    /data4/mct/raw/20211110_DC251/DC251.PT.PET_07_...      12:29:00   
 48    /data4/mct/raw/20211110_DC251/DC251.PT.PET_07_...      12:29:00   
 59    /data4/mct/raw/20211110_DC251/DC251.PT.PET_07_...      12:29:00   
 72    /data4/mct/raw/20211110_DC251/DC251.PT.PET_07_...      12:29:00   
 101   /data4/mct/raw/20211110_DC251/DC251.PT.PET_07_...      12:29:00   
 ...         

In [ ]:
#modify 3D clouds to recon
df_index=0
for summary in summaries: 
    summary_dict = summary.to_dict('records')
    for i in range(len(summary_dict)):
        x = summary_dict[i]['ThreeD_Cloud'].find('3D_Clouds_nii')
        fn_cloud1 = summary_dict[i]['ThreeD_Cloud'][0:x] + 'MOLAR_Recon_NAC_nii_suv_resize'
        x = x+13
        y = summary_dict[i]['ThreeD_Cloud'].find('3D_cloud')
        fn_cloud2 =  summary_dict[i]['ThreeD_Cloud'][x:y] + 'recon_suv_rz.nii'
        summary_dict[i]['ThreeD_Cloud'] = fn_cloud1 + fn_cloud2
    summaries[df_index] = pd.DataFrame(summary_dict)
    df_index = df_index + 1


In [1]:
# Define new transforms (especially define new num_samples) for testing
pet_transforms2 = monai.transforms.Compose([
    CreateImageStackd(keys="DataFrame", image_key=image_key, spatial_size=SIZE),
    RandSamplePETd(keys=image_key, meta_keys=meta_keys, num_samples=32, meta_data_key='DataFrame'), 
    ComputeRelativeMotiond(keys=['VICRA_ref','VICRA_mov'], output_key='VICRA_rel'),
    monai.transforms.DeleteItemsd(keys=["DataFrame", "ScanStart", "VICRA_ref", "VICRA_mov", image_key]),
    monai.transforms.ToTensord(
        keys=["ScanStart_ref", "ScanStart_mov", "VICRA_rel"]
    )
])

# Create dataset and dataloader 
test_ds = monai.data.Dataset(
    data=test_list, 
    transform=pet_transforms2, 
)
test_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=1, 
    collate_fn=monai.data.list_data_collate
)

# Rebuild a dataframe with reference and moving clouds paths and VICRA 
ref_clouds=[]
mov_clouds=[]
pairs=[]
vicra=[]

i=0
for test_data in test_loader:
    summary=summaries[i]
    for j in range(len(test_data['ThreeD_Cloud_ref'])):
        idx=int(test_data['ScanStart_ref'][j]-3600)
        ref_clouds.append(summary['ThreeD_Cloud'][idx])
        
        idx2=int(test_data['ScanStart_mov'][j]-3600)
        mov_clouds.append(summary['ThreeD_Cloud'][idx2])
        
        pairs.append(np.array([test_data['ScanStart_ref'][j], test_data['ScanStart_mov'][j]]))
        
        vicra.append(test_data['VICRA_rel'][j].float())
        
    i+=1
    
new_df_test=pd.DataFrame()

new_df_test['ThreeD_Cloud_ref']=ref_clouds
new_df_test['ThreeD_Cloud_mov']=mov_clouds
new_df_test['pairs']=pairs
new_df_test['VICRA_rel']=vicra

# Back to our regular testing process 
KEYS = ['ThreeD_Cloud_ref','ThreeD_Cloud_mov'] 

train_transforms = Compose([
    LoadImaged(keys=KEYS, reader='NibabelReader', as_closest_canonical=False),
    monai.transforms.EnsureChannelFirstd(keys=KEYS), # previously AddChanneld
    Orientationd(keys=KEYS, axcodes='RAS'),
    ToTensord(keys=KEYS)    
])

y_list_all_test=[]
predictions_all_test=[]

##building testing dataloader
test_dict = new_df_test.to_dict('records')

for i in range(len(test_dict)):
    x = test_dict[i]['ThreeD_Cloud_ref'].find('3D_Clouds_nii')
    fn_cloud1 = test_dict[i]['ThreeD_Cloud_ref'][0:x] + 'MOLAR_Recon_NAC_nii_suv_resize'
    x = x+13
    y = test_dict[i]['ThreeD_Cloud_ref'].find('3D_cloud')
    fn_cloud2 =  test_dict[i]['ThreeD_Cloud_ref'][x:y] + 'recon_suv_rz.nii'
    test_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2

    x = test_dict[i]['ThreeD_Cloud_mov'].find('3D_Clouds_nii')
    fn_cloud1 = test_dict[i]['ThreeD_Cloud_mov'][0:x] + 'MOLAR_Recon_NAC_nii_suv_resize'
    x = x+13
    y = test_dict[i]['ThreeD_Cloud_mov'].find('3D_cloud')
    fn_cloud2 =  test_dict[i]['ThreeD_Cloud_mov'][x:y] + 'recon_suv_rz.nii'
    test_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2

# Create the Dataset
ds_test = monai.data.CacheDataset(data=test_dict, transform=train_transforms)
# Create the DataLoader
test_loader = monai.data.DataLoader(ds_test, batch_size=1, num_workers=2, collate_fn=list_data_collate)

#calculate loss function and network output
saved_model_path = os.path.join(MODEL_PATH,'PETRegNet-epoch=4679-val_loss=0.204.ckpt')
loaded_model = PETRegNet.load_from_checkpoint(saved_model_path)

loaded_model.eval()
loaded_model.to(device)
loss = 0
loss_list = list()
time_list = list()
prediction_list = list()
y_list = list()

i=0
for test_data in test_loader:
    x1 = test_data['ThreeD_Cloud_ref'].to(device)
    x2 = test_data['ThreeD_Cloud_mov'].to(device)
    y_temp = test_data['VICRA_rel'].cpu().numpy()
    y_temp = y_temp.flatten()
    y = RotTransMatrix_6Params(y_temp,1)
    y_test = loaded_model(x1, x2).detach().cpu().numpy()

    loss = np.mean(np.square(y-y_test))
    i = i + 1
    loss_list.append(loss)
    prediction_list.append(y_test)
    y_list.append(y)
    predictions_all_test.append(prediction_list)

y_list_all_test.append(y_list)

df_loss = pd.DataFrame()
df_loss['Loss'] = loss_list
df_loss.describe()

NameError: name 'monai' is not defined

In [20]:
len(y)

TypeError: object of type 'int' has no len()

In [23]:
#calculate loss function and network output
saved_model_path = os.path.join(MODEL_PATH,'PETRegNet-epoch=4679-val_loss=0.204.ckpt')
loaded_model = PETRegNet.load_from_checkpoint(saved_model_path)

loaded_model.eval()
loaded_model.to(device)
loss = 0
loss_list = list()
time_list = list()
prediction_list = list()
y_list = list()

i=0
for test_data in test_loader:
    x1 = test_data['ThreeD_Cloud_ref'].to(device)
    x2 = test_data['ThreeD_Cloud_mov'].to(device)
    y_temp = test_data['VICRA_rel'].cpu().numpy()
    y_temp = y_temp.flatten()
    y = RotTransMatrix_6Params(y_temp,1)
    y_test = loaded_model(x1, x2).detach().cpu().numpy()

    loss = np.mean(np.square(y-y_test))
    i = i + 1
    loss_list.append(loss)
    prediction_list.append(y_test)
    y_list.append(y)
    predictions_all_test.append(prediction_list)

y_list_all_test.append(y_list)

df_loss = pd.DataFrame()
df_loss['Loss'] = loss_list
df_loss.describe()

,Loss
count,8.000000
mean,0.323140
std,0.189787
min,0.061963
25%,0.170094
50%,0.314212
75%,0.482968
max,0.571546


In [22]:
y_temp

array([[ 0.9984261 , -0.02469312,  0.05035143, -7.3230486 ,  0.02397848,
         0.9996037 ,  0.01473264, -3.7240846 , -0.05069525, -0.01350225,
         0.9986228 , -5.040263  ]], dtype=float32)